# Project 2 Jupyter Notebook
## by Qingyue Li

In [ ]:
## scientific question
## background (harvest, function)
## hypothesis (if-then)
## what analyses were done and how the data was downloaded for the project

In [ ]:
## package loading
## definition

from Bio import SeqIO
import numpy as np
import matplotlib.pyplot as plt
from Bio import pairwise2

from Bio.Blast import NCBIWWW
from Bio.Blast.Applications import NcbiblastnCommandline 
from Bio.Blast import NCBIXML




In [20]:
#BLASTP on 3 allergenic protein sequences

from Bio.Blast import NCBIWWW
sequence_data = open("allergen_sequence.fasta").read() 
sequence_data 

result_handle = NCBIWWW.qblast("blastp", "nr", sequence_data, entrez_query="txid3702[ORGN]") 
result_handle

with open('results.xml', 'w') as save_file: 
    blast_results = result_handle.read() 
    save_file.write(blast_results)

from Bio.Blast.Applications import NcbiblastnCommandline     
blastn_cline = NcbiblastnCommandline(query = "search.fasta", db = "alun", 
                                     outfmt = 5, out = "results.xml") 
stdout, stderr = blastn_cline()

from Bio.Blast import NCBIXML
E_VALUE_THRESH = 1e-20 
for record in NCBIXML.parse(open("results.xml")): 
    if record.alignments: 
        print("\n") 
        print("query: %s" % record.query[:100]) 
        for align in record.alignments: 
            for hsp in align.hsps:  
                 if hsp.expect < E_VALUE_THRESH: 
                    print("match: %s " % align.title[:100])

ApplicationError: Non-zero return code 127 from 'blastn -out results.xml -outfmt 5 -query search.fasta -db alun', message '/bin/sh: blastn: command not found'

In [ ]:
# protein structure analysis

from Bio.PDB import *
import nglview as nv
import ipywidgets

pdb_parser = PDBParser()
structure = pdb_parser.get_structure("PHA-L", "Data/1FAT.pdb")
view = nv.show_biopython(structure)

#accessing
mmcif_dict = MMCIFDict.MMCIFDict("fa/1fat.cif")
len(mmcif_dict) 

# .get_residues() method in a loop
for model in structure:
    for residue in model.get_residues():
        print(residue)
        
#build polypeptides
polypeptide_builder = CaPPBuilder()
counter = 1
for polypeptide in polypeptide_builder.build_peptides(structure):
    seq = polypeptide.get_sequence()
    print(f"Sequence: {counter}, Length: {len(seq)}")
    print(seq)
    counter += 1
    

#residue analysis    
from Bio.SeqUtils.ProtParam import ProteinAnalysis
analyzed_seq = ProteinAnalysis(str(seq))

#molecular weight
analyzed_seq.molecular_weight()



## need to make 3 heatplots for comparison between 3 sequences

In [ ]:
# pairwise for heatlplot

Fasta = list(SeqIO.parse("allergen_sequence.fasta", "fasta"))

size = len(Fasta)

#Initiate variable i and j for for loop
i = 0
j = 0

#Initiate an empy numpy matrix that is the length of the fasta file
alignments_score = np.empty(shape=(size,size))


#Nested for loop to do pairwise sequence alignment for each pair of kinases
def pairwise_seq_heatmap(my_numpy_array, size):
    for i in range(size):
        for j in range(size):
        # Define two sequences to be aligned
            X = Fasta[i].seq
            Y = Fasta[j].seq
            print(Fasta[i].id)

        # Get a list of the global alignments between the two sequences ACGGGT and ACG
        # No parameters. Identical characters have score of 1, else 0.
        # No gap penalties.
            my_numpy_array[i,j] = int(pairwise2.align.globalxx(X, Y, score_only=True))

    return my_numpy_array

# Check that the for loop correctly created the matrix
My_alignment_Scores = pairwise_seq_heatmap(alignments_score, size)

#Create subplot
fig, ax = plt.subplots()

plt.imshow(alignments_score)

# Construct the heat map to show the axes
im = ax.imshow(alignments_score)

# Add a colorbar to the right of the heatmap
cbar = ax.figure.colorbar(im)

# Show the plot
plt.show()

# Gene Sequence Sources

Arachis hypogaea (peanut)
Arachis hypogaea main allergen Ara h1 mRNA, complete cds
https://www.ncbi.nlm.nih.gov/nuccore/193850560
    
Pistacia vera (pistachio)
Pistacia vera Pis v 1 allergen 2S albumin mRNA, complete cds
https://www.ncbi.nlm.nih.gov/nuccore/110349080

Anacardium occidentale (cashew)
Anacardium occidentale allergen Ana o 2 mRNA, partial cds
https://www.ncbi.nlm.nih.gov/nuccore/25991542